In [23]:
# Importação de bibliotecas

import os
from dotenv import load_dotenv, find_dotenv
import anthropic
from openai import OpenAI
from langchain_groq import ChatGroq
from IPython.display import Markdown, display, update_display

In [24]:
# Load environment variables in a file called .env

load_dotenv(find_dotenv())
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if anthropic_api_key:
    print(f"Anthropic API key exists and begings with {anthropic_api_key[:7]}")
else:
    print("No Anthropic API key found. Please set it in a .env file.")

if openai_api_key:
    print(f"OpenAI API key exists and begings with {openai_api_key[:8]}")
else:
    print("No OpenAI API key found. Please set it in a .env file.")

if groq_api_key:
    print(f"Groq API key exists and begings with {groq_api_key[:8]}")
else:
    print("No Groq API key found. Please set it in a .env file.")

Anthropic API key exists and begings with sk-ant-
OpenAI API key exists and begings with sk-proj-
Groq API key exists and begings with gsk_lVJs


In [30]:
# Connect to Anthropic API
claude = anthropic.Anthropic(api_key=anthropic_api_key)
openai = OpenAI(api_key=openai_api_key)
# groq = ChatGroq(api_key=groq_api_key)


## Asking LLMs to tell a joke

it turns out that LLMs don't do a great job of telling jokes! Let's compare a few models. Later we will be putting LLMs to better use!

### What information is included in the API

**Typycally we'll pass to the API:**
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is tipically between 0 and 1; higher for more random output; lower for more focused and deterministic.



In [ ]:
system_message = "You are an assistant that is great at telling jokes. Ans"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists. in Portguese"

In [11]:
prompts = [
    {"role":"system","content":system_message},
    {"role":"user", "content": user_prompt}
]

In [14]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role":"user","content":user_prompt}
    ]
)

print(message.content[0].text)

Aqui está uma piada para cientistas de dados:

Por que o cientista de dados ficou desempregado?

Porque ele não conseguia fazer uma boa *previsão* do mercado de trabalho! 

*ba dum tss* 🥁

Alternativa:

Qual é o lanche preferido do cientista de dados?

*Cookie*s... especialmente os de terceiros! 🍪

(Trocadilho com "third-party cookies" - cookies de terceiros, um termo comum em dados web)


## Back to OpenAI with a serious question

In [15]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [31]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you can evaluate the following criteria:

## 1. Nature of the Problem
- **Text-Based**: Is the problem primarily text-based, involving natural language processing (NLP) tasks such as understanding, generating, or summarizing text?
- **Complexity**: Does the problem require understanding context, sentiment, or nuanced language?

## 2. Data Availability
- **Quality of Data**: Do you have access to high-quality, relevant textual data for training or fine-tuning the LLM?
- **Quantity of Data**: Is there enough data available to train or fine-tune an LLM effectively?

## 3. Business Objectives
- **Clear Goals**: Are the objectives specific and measurable? For example, improving customer support response times or generating marketing content.
- **Value Addition**: Will using an LLM provide significant value, such as increased efficiency, cost savings, or enhanced customer experience?

## 4. Technical Feasibility
- **Infrastructure**: Do you have the necessary computational resources to run an LLM, or can you leverage cloud-based solutions?
- **Integration**: Can the LLM be integrated into your existing systems and workflows without excessive disruption?

## 5. User Interaction
- **User-Centric**: Will users interact with the solution in a meaningful way, such as through chatbots, virtual assistants, or content generation tools?
- **Feedback Loop**: Is there a mechanism for users to provide feedback to improve the model’s performance over time?

## 6. Ethical Considerations
- **Bias and Fairness**: Are there concerns about bias in the data that could lead to unfair or unethical outcomes?
- **Transparency**: Is it important for the solution to be interpretable and explainable to users and stakeholders?

## 7. Regulatory Compliance
- **Data Privacy**: Are there legal and regulatory considerations regarding data usage, especially in sensitive industries like healthcare or finance?
- **Compliance**: Does the solution align with industry regulations and standards?

## Conclusion
If the business problem fits well within these criteria, it is likely suitable for an LLM solution. Conducting a thorough assessment will help ensure that you make an informed decision about leveraging LLM technology in your business context.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [36]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [37]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [38]:
call_gpt()

'Oh, a simple "hi"? How original. Can\'t we aim for a little more creativity here?'

In [45]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        # print(gpt)
        messages.append({"role": "assistant", "content": claude_message})
        # print(claude_message)
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [46]:
call_claude()

'Hello! How are you doing today?'

In [42]:
call_gpt()

'Oh great, another "hi." How original. What do you want?'

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)